# Hypercomplex Numbers

*simplified version*

In [28]:
from abc import ABC, abstractmethod
from numbers import Number
from fractions import Fraction

In [126]:
class Hypercomplex(ABC, Number):

    def __init__(self, real: Number, imag: Number):
        self._real = real
        self._imag = imag

    @property
    def real(self):
        return self._real

    @property
    def imag(self):
        return self._imag

    def __repr__(self):
        if isinstance(self.real, (int, float, complex)):
            return f"{self.__class__.__name__}({self.real}, {self.imag})"
        elif isinstance(self.real, Fraction):
            return f"{self.__class__.__name__}('{self.real}', '{self.imag}')"
        else:
            return f"{self.__class__.__name__}({repr(self.real)}, {repr(self.imag)})"

    def __add__(self, other):
        return self.__class__(self.real + other.real, self.imag + other.imag)

    def __sub__(self, other):
        return self.__class__(self.real - other.real, self.imag - other.imag)

    def __mul__(self, other):
        if isinstance(other, Number) and not isinstance(other, Hypercomplex):
            oth = self.__class__(other)
        else:
            oth = other
        n = self.order
        k = oth.order
        if n == k:
            a, b, c, d = self.real, self.imag, oth.real, oth.imag
            real_part = a * c - d.conjugate() * b
            imag_part = d * a + b * c.conjugate()
            return Zi(real_part, imag_part)
        elif n > k:
            return self * oth.increase_order(n)
        else:
            return self.increase_order(k) * oth
            
    def __hash__(self):
        return hash((self.real, self.imag, type(self)))

    def __len__(self):
        return 2

    def __getitem__(self, index):
        if isinstance(index, int):
            if index == 0:
                return self.real
            elif index == 1:
                return self.imag
            else:
                raise IndexError(f"Index {index} out of range")
        else:
            raise TypeError(f"Index {index} must be a non-negative integer")

    def __iter__(self):
        return iter((self.real, self.imag))

    def __eq__(self, other):
        if type(self) == type(other):
            return self.real == other.real and self.imag == other.imag
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)
    
    def __neg__(self):
        return self.__class__(-self.real, -self.imag)
        
    def conj(self):
        return self.__class__(self.real, -self.imag)

    def __abs__(self) -> float:
        return sqrt(self.norm)

    def __pos__(self):
        return self

    @property
    def norm(self):
        return self.real * self.real + self.imag * self.imag

    @property
    def order(self):
        def aux(x, d):
            if isinstance(x, (int, float, complex, Fraction)):
                return d
            else:
                return aux(x.real, d + 1)
        return aux(self.real, 1)

    @property
    def dim(self):
        return 2 ** self.order

    @property
    def first(self):
        if isinstance(self.real, (int, float, complex, Fraction)):
            return self.real
        elif isinstance(self.real, Hypercomplex):
            return self.real.first
        else:
            raise Exception(f"Cannot obtain a real component from {self}")

    def zero(self, order=1):
        if isinstance(order, int) and order >= 1:
            if order == 1:
                return self.__class__(0, 0)
            else:
                d = order - 1
                return self.__class__(self.__class__.zero(d), self.__class__.zero(d))
        else:
            raise Exception(f"Cannot create a zero with {order}")

    def increase_order(self, d: int):
        if isinstance(d, int) and d >= 1:
            n = self.order
            if n == d:
                return self
            elif n < d:
                return self.__class__(self, self.__class__.zero(n)).increase_order(d)
            else:
                raise Exception(f"Should not reach this line, {self = }, {d = }")
        else:
            raise ValueError(f"{d = }, is not an integer >= 1")

In [127]:
class Z(Hypercomplex):

    def __init__(self, real=0, imag=0):
        super().__init__(real, imag)

In [128]:
class Zi(Hypercomplex):

    def __init__(self, real=0, imag=0):

        re = real
        im = imag

        if isinstance(real, (int, float)):
            re = round(real)

        if isinstance(imag, (int, float)):
            im = round(imag)
        
        super().__init__(re, im)

In [129]:
class Qi(Hypercomplex):

    _max_denominator = 1_000_000

    def __init__(self, real=Fraction(0, 1), imag=Fraction(0, 1)):

        re = real
        im = imag

        if isinstance(real, (int, float, str)):
            re = Fraction(real)
        
        if isinstance(imag, (int, float, str)):
            im = Fraction(imag)
        
        super().__init__(re, im)

In [171]:
print(f"{Zi() = }")
print(f"{Zi(7) = }")
print(f"{Zi(1, 2) = }")
print(f"{Zi(Zi(1, 2), Zi(3, 4)) = }")
print(f"{Zi(1, 2) + Zi(3, 4) = }")
print(f"{Zi(1, 2) - Zi(3, 4) = }")
print(f"{Zi(1, 2) * Zi(3, 4) = }")
print(f"{(1 + 2j) * (3 + 4j) = }")
print(f"{Zi(1, 2).norm = }")
print(f"{-Zi(1, 2) = }")
print(f"{Zi(1, 2).conj() = }")
# print(f"{Zi(1, 2).abs = }")
print(f"{Zi(1, 2).order = }")
print(f"{Zi(1, 2).real = }")
print(f"{Zi(1, 2).imag = }")
print(f"{Zi(1, 2).dim = }")
print(f"{Zi(1, 2).first = }")

Zi() = Zi(0, 0)
Zi(7) = Zi(7, 0)
Zi(1, 2) = Zi(1, 2)
Zi(Zi(1, 2), Zi(3, 4)) = Zi(Zi(1, 2), Zi(3, 4))
Zi(1, 2) + Zi(3, 4) = Zi(4, 6)
Zi(1, 2) - Zi(3, 4) = Zi(-2, -2)
Zi(1, 2) * Zi(3, 4) = Zi(-5, 10)
(1 + 2j) * (3 + 4j) = (-5+10j)
Zi(1, 2).norm = 5
-Zi(1, 2) = Zi(-1, -2)
Zi(1, 2).conj() = Zi(1, -2)
Zi(1, 2).order = 1
Zi(1, 2).real = 1
Zi(1, 2).imag = 2
Zi(1, 2).dim = 2
Zi(1, 2).first = 1


In [181]:
print(f"{Qi() = }")
print(f"{Qi('7/8') = }")
print(f"{Qi(3, '2/3') = }")
print(f"{Qi(Qi('1/4', '2/3'), Qi('3/5', '4/6')) = }")
print(f"{Qi('1/4', '2/5') + Qi('3/5', '4/5') = }")
print(f"{Qi('1/4', '2/5') - Qi('3/5', '4/5') = }")
print(f"{Qi('1/4', '2/5') * Qi('3/5', '4/5') = }")
print(f"{(0.25 + 0.4j) * (0.6 + 0.8j) = }")
print(f"{Qi('2/3', '1/2').norm = }")
print(f"{Qi('1/3', '2/4').order = }")
print(f"{Qi('1/3', '2/4').real = }")
print(f"{Qi('1/3', '2/4').imag = }")
print(f"{Qi('1/3', '2/4').dim = }")
print(f"{Qi('1/3', '2/4').first = }")

print(f"\n{Qi(Qi('1/4', '2/3'), Qi('3/5', '4/6')).real = }")
print(f"{Qi(Qi('1/4', '2/3'), Qi('3/5', '4/6')).imag = }")
print(f"{Qi(Qi('1/4', '2/3'), Qi('3/5', '4/6')).dim = }")
print(f"{Qi(Qi('1/4', '2/3'), Qi('3/5', '4/6')).order = }")
print(f"{Qi(Qi('1/4', '2/3'), Qi('3/5', '4/6')).norm = }")
print(f"{Qi(Qi('1/4', '2/3'), Qi('3/5', '4/6')).conj() = }")

Qi() = Qi('0', '0')
Qi('7/8') = Qi('7/8', '0')
Qi(3, '2/3') = Qi('3', '2/3')
Qi(Qi('1/4', '2/3'), Qi('3/5', '4/6')) = Qi(Qi('1/4', '2/3'), Qi('3/5', '2/3'))
Qi('1/4', '2/5') + Qi('3/5', '4/5') = Qi('17/20', '6/5')
Qi('1/4', '2/5') - Qi('3/5', '4/5') = Qi('-7/20', '-2/5')
Qi('1/4', '2/5') * Qi('3/5', '4/5') = Zi('-17/100', '11/25')
(0.25 + 0.4j) * (0.6 + 0.8j) = (-0.17000000000000007+0.44j)
Qi('2/3', '1/2').norm = Fraction(25, 36)
Qi('1/3', '2/4').order = 1
Qi('1/3', '2/4').real = Fraction(1, 3)
Qi('1/3', '2/4').imag = Fraction(1, 2)
Qi('1/3', '2/4').dim = 2
Qi('1/3', '2/4').first = Fraction(1, 3)

Qi(Qi('1/4', '2/3'), Qi('3/5', '4/6')).real = Qi('1/4', '2/3')
Qi(Qi('1/4', '2/3'), Qi('3/5', '4/6')).imag = Qi('3/5', '2/3')
Qi(Qi('1/4', '2/3'), Qi('3/5', '4/6')).dim = 4
Qi(Qi('1/4', '2/3'), Qi('3/5', '4/6')).order = 2
Qi(Qi('1/4', '2/3'), Qi('3/5', '4/6')).norm = Zi('-1679/3600', '17/15')
Qi(Qi('1/4', '2/3'), Qi('3/5', '4/6')).conj() = Qi(Qi('1/4', '2/3'), Qi('-3/5', '-2/3'))
